In [1]:
import sys
import cmdstanpy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import pandas as pd
import json

sys.path.insert(0, '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF')
sys.path.insert(0, '/home/garren/Documents/MEng/Code/Latest_results/HPC Files')

from Post_procs import PostProcess
from All_code import subsets
import os

from IPython.display import clear_output
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
functional_groups = np.array(['all'])
include_clusters = True
variance_known = True
inf_type = 'MAP'

In [ ]:
post_obj = PostProcess(functional_groups=functional_groups, 
                       include_clusters=include_clusters, 
                       variance_known=variance_known, 
                       inf_type=inf_type)

In [ ]:
data_dict = post_obj.get_MC()

In [ ]:
rec_dict = post_obj.get_reconstructed_errors()

In [ ]:
err_metrics = post_obj.compute_error_metrics(data_dict=data_dict, is_testing=True)

In [ ]:
rec_err_metrics = post_obj.compute_error_metrics(data_dict=rec_dict, is_testing=False)

In [ ]:
post_obj.path

In [ ]:
bins = 50
er_MC = np.abs(data_dict['y_MC'] - data_dict['y_exp'])
er_UNIFAC = np.abs(data_dict['y_UNIFAC'] - data_dict['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC[er_MC > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = f'{post_obj.path}/Error_plots/Testing'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC, bins=bin_edges, color='b', alpha=0.5, label='MC')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.15, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(data_dict['y_MC'], data_dict['y_exp'], '.b', label='MC', alpha=0.5)
ax[1].plot(data_dict['y_UNIFAC'], data_dict['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.15, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(data_dict['y_exp']), max(data_dict['y_MC']), max(data_dict['y_UNIFAC']))
min_lim = min(min(data_dict['y_exp']), min(data_dict['y_MC']), min(data_dict['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
bins = 50
er_MC = np.abs(rec_dict['y_MC'] - rec_dict['y_exp'])
er_UNIFAC = np.abs(rec_dict['y_UNIFAC'] - rec_dict['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC[er_MC > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = f'{post_obj.path}/Error_plots/Training'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC, bins=bin_edges, color='b', alpha=0.5, label='MC')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.15, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(rec_dict['y_MC'], rec_dict['y_exp'], '.b', label='MC', alpha=0.5)
ax[1].plot(rec_dict['y_UNIFAC'], rec_dict['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.15, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(rec_dict['y_exp']), max(rec_dict['y_MC']), max(rec_dict['y_UNIFAC']))
min_lim = min(min(rec_dict['y_exp']), min(rec_dict['y_MC']), min(rec_dict['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
idx1 = np.sum((err_metrics['IUPAC', 'Component 1'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)
idx2 = np.sum((err_metrics['IUPAC', 'Component 2'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)

train_idx = np.column_stack([idx1, idx2])

top_keys = ['UNIFAC', 'MC']
bottom_keys = ['MAE', 'RMSE', 'MARE']

num_top = len(top_keys)
num_bottom = len(bottom_keys)

A = np.nan*np.zeros((num_bottom*num_top, post_obj.c_all.shape[0], post_obj.c_all.shape[0]))

path = f'{post_obj.path}/Error_plots/Testing'
if not os.path.exists(path):
    os.makedirs(path)

for i in range(num_top):
    for j in range(num_bottom):
        A[i*num_bottom + j, train_idx[:,1], train_idx[:,0]] = err_metrics[top_keys[i], bottom_keys[j]][:-1].to_numpy()
        fig, ax = plt.subplots(figsize=(20, 20))
        cax = ax.imshow(A[i*num_bottom + j])
        ax.set_xticks(ticks=np.arange(post_obj.c_all.shape[0]), labels=post_obj.c_all, rotation=90)
        ax.set_yticks(ticks=np.arange(post_obj.c_all.shape[0]), labels=post_obj.c_all)
        ax.set_aspect('equal')

        cbar = fig.colorbar(cax, ax=ax, orientation='vertical', shrink=0.5)

        file_name = f'{path}/{top_keys[i]}_{bottom_keys[j]}_sparse.png'

        fig.savefig(file_name, dpi=400)

        plt.close(fig)
        clear_output(wait=False)

In [ ]:
idx1 = np.sum((rec_err_metrics['IUPAC', 'Component 1'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)
idx2 = np.sum((rec_err_metrics['IUPAC', 'Component 2'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)

idx = np.column_stack([idx1, idx2])

top_keys = ['UNIFAC', 'MC']
bottom_keys = ['MAE', 'RMSE']

num_top = len(top_keys)
num_bottom = len(bottom_keys)

A = np.nan*np.zeros((num_bottom*num_top, post_obj.c_all.shape[0], post_obj.c_all.shape[0]))

path = f'{post_obj.path}/Error_plots/Training'
if not os.path.exists(path):
    os.makedirs(path)

for i in range(num_top):
    for j in range(num_bottom):
        A[i*num_bottom + j, idx[:,1], idx[:,0]] = rec_err_metrics[top_keys[i], bottom_keys[j]][:-1].to_numpy()
        fig, ax = plt.subplots(figsize=(20, 20))
        cax = ax.imshow(A[i*num_bottom + j])
        ax.set_xticks(ticks=np.arange(post_obj.c_all.shape[0]), labels=post_obj.c_all, rotation=90)
        ax.set_yticks(ticks=np.arange(post_obj.c_all.shape[0]), labels=post_obj.c_all)
        ax.set_aspect('equal')

        cbar = fig.colorbar(cax, ax=ax, orientation='vertical', shrink=0.5)

        file_name = f'{path}/{top_keys[i]}_{bottom_keys[j]}_sparse.png'

        fig.savefig(file_name, dpi=400)

        plt.close(fig)
        clear_output(wait=False)

In [ ]:
excel_err_file = f'{post_obj.path}/Errors_Metrics.xlsx'
with pd.ExcelWriter(excel_err_file, engine='xlsxwriter') as writer:
    err_form = err_metrics.copy()
    err_form.iloc[:, -6:] = err_form.iloc[:, -6:].round(2)
    err_form.to_excel(writer, sheet_name='Testing')

    rec_err_form = rec_err_metrics.copy()
    rec_err_form.iloc[:, -4:] = rec_err_form.iloc[:, -4:].round(2)
    rec_err_form.to_excel(writer, sheet_name='Training')

In [3]:
post_obj = PostProcess(functional_groups=functional_groups, 
                       include_clusters=False, 
                       variance_known=variance_known, 
                       inf_type=inf_type)

data_dict_c_0 = post_obj.get_MC()
rec_dict_c_0 = post_obj.get_reconstructed_errors()
err_metrics_c_0 = post_obj.compute_error_metrics(data_dict=data_dict_c_0, is_testing=True)
rec_err_metrics_c_0 = post_obj.compute_error_metrics(data_dict=rec_dict_c_0, is_testing=False)
A_c_0 = post_obj.get_pred_param_matrix()

In [4]:
post_obj = PostProcess(functional_groups=functional_groups, 
                       include_clusters=True, 
                       variance_known=variance_known, 
                       inf_type=inf_type)

data_dict_c_1 = post_obj.get_MC()
rec_dict_c_1 = post_obj.get_reconstructed_errors()
err_metrics_c_1 = post_obj.compute_error_metrics(data_dict=data_dict_c_1, is_testing=True)
rec_err_metrics_c_1 = post_obj.compute_error_metrics(data_dict=rec_dict_c_1, is_testing=False)
A_c_1 = post_obj.get_pred_param_matrix()

Faulty csv file: /home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Include_clusters_True/Variance_known_True/MAP/3/Hybrid_PMF_include_clusters_True_variance_known_True-20240725053021.csv
Skipping...
Faulty csv file: /home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Include_clusters_True/Variance_known_True/MAP/4/Hybrid_PMF_include_clusters_True_variance_known_True-20240724215804.csv
Skipping...
Faulty csv file: /home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Include_clusters_True/Variance_known_True/MAP/3/Hybrid_PMF_include_clusters_True_variance_known_True-20240725053021.csv
Skipping...
Faulty csv file: /home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Include_clusters_True/Variance_known_True/MAP/4/Hybrid_PMF_include_clusters_True_variance_known_True-20240724215804.csv
Skipping...
Faulty csv file: /home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybri

In [ ]:
bins = 50
er_MC_c_0 = np.abs(data_dict_c_0['y_MC'] - data_dict_c_0['y_exp'])
er_MC_c_1 = np.abs(data_dict_c_1['y_MC'] - data_dict_c_1['y_exp'])
er_UNIFAC = np.abs(data_dict_c_0['y_UNIFAC'] - data_dict_c_0['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC_c_0[er_MC_c_0 > bin_edges[-1]] = bin_edges[-1]
er_MC_c_1[er_MC_c_1 > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Error_plots/Testing'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC_c_0, bins=bin_edges, color='b', alpha=0.5, label='MC - No clusters')
ax[0].hist(er_MC_c_1, bins=bin_edges, color='r', alpha=0.5, label='MC - With clusters')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(data_dict_c_0['y_MC'], data_dict_c_0['y_exp'], '.b', label='MC - No clusters', alpha=0.5)
ax[1].plot(data_dict_c_1['y_MC'], data_dict_c_1['y_exp'], '.r', label='MC - With clusters', alpha=0.5)
ax[1].plot(data_dict_c_0['y_UNIFAC'], data_dict_c_0['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(data_dict_c_0['y_exp']), max(data_dict_c_0['y_MC']), max(data_dict_c_1['y_MC']), max(data_dict_c_0['y_UNIFAC']))
min_lim = min(min(data_dict_c_0['y_exp']), min(data_dict_c_0['y_MC']), min(data_dict_c_1['y_MC']), min(data_dict_c_0['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
bins = 50
er_MC_c_0 = np.abs(rec_dict_c_0['y_MC'] - rec_dict_c_0['y_exp'])
er_MC_c_1 = np.abs(rec_dict_c_1['y_MC'] - rec_dict_c_1['y_exp'])
er_UNIFAC = np.abs(rec_dict_c_0['y_UNIFAC'] - rec_dict_c_0['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC_c_0[er_MC_c_0 > bin_edges[-1]] = bin_edges[-1]
er_MC_c_1[er_MC_c_1 > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Error_plots/Training'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC_c_0, bins=bin_edges, color='b', alpha=0.5, label='MC - No clusters')
ax[0].hist(er_MC_c_1, bins=bin_edges, color='r', alpha=0.5, label='MC - With clusters')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(rec_dict_c_0['y_MC'], rec_dict_c_0['y_exp'], '.b', label='MC - No clusters', alpha=0.5)
ax[1].plot(rec_dict_c_1['y_MC'], rec_dict_c_1['y_exp'], '.r', label='MC - With clusters', alpha=0.5)
ax[1].plot(rec_dict_c_0['y_UNIFAC'], rec_dict_c_0['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(rec_dict_c_0['y_exp']), max(rec_dict_c_0['y_MC']), max(rec_dict_c_1['y_MC']), max(rec_dict_c_0['y_UNIFAC']))
min_lim = min(min(rec_dict_c_0['y_exp']), min(rec_dict_c_0['y_MC']), min(rec_dict_c_1['y_MC']), min(rec_dict_c_0['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/2D_plots/Testing'
excel_UNI_sheet = 'Testing_Plots'
_, Idx_known = post_obj.get_excess_enthalpy()
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(data_dict_c_0['c1'].astype(str), ' + '), data_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(data_dict_c_0['c1'], ' + '), data_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    y_idx = exp_mix == unique_mix[j]
    UNIFAC_idx = UNIFAC_mix == unique_mix[j]
    yy = data_dict_c_0['y_exp'][y_idx]
    yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    x_y = data_dict_c_0['x'][y_idx]
    T_y = data_dict_c_0['T'][y_idx]
    c1 = data_dict_c_0['c1'][y_idx][0]
    c2 = data_dict_c_0['c2'][y_idx][0]

    x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

    p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

    p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

    T_uniq = np.unique(T_UNIFAC)
    for i in range(len(T_uniq)):
        if not os.path.exists(f'{path}/{j}_{i}.png'):
            print(f'{j+1} out of {len(unique_mix)} mixtures')
            print(f'{i+1} out of {len(T_uniq)} temperatures')
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT

            fig, ax = plt.subplots()
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], '-g', label='UNIFAC')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], '-b', label='Mean MC - No clusters')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], '-r', label='Mean MC - With clusters')
            ax.plot(x_y[T_y_idx], yy[T_y_idx], '.k', label='Experimental Data')
            ax.set_xlabel('Composition of Compound 1 [mol/mol]')
            ax.set_ylabel('Excess Enthalpy [J/mol]')
            ax.set_title(f'(1) {c1} + (2) {c2} at {T_uniq[i]:.2f} K')
            ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            fig.savefig(f'{path}/{j}_{i}.png', dpi=300)
            plt.close(fig)
            clear_output(wait=False)

In [ ]:
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/3D_plots/Testing'
excel_UNI_sheet = 'Testing_Plots'
_, Idx_known = post_obj.get_excess_enthalpy()
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(data_dict_c_0['c1'].astype(str), ' + '), data_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(data_dict_c_0['c1'], ' + '), data_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    if not os.path.exists(f'{path}/{j}.png'):
        print(f'{j+1} out of {len(unique_mix)} mixtures')
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        y_idx = exp_mix == unique_mix[j]
        UNIFAC_idx = UNIFAC_mix == unique_mix[j]
        yy = data_dict_c_0['y_exp'][y_idx]
        yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        x_y = data_dict_c_0['x'][y_idx]
        T_y = data_dict_c_0['T'][y_idx]
        c1 = data_dict_c_0['c1'][y_idx][0]
        c2 = data_dict_c_0['c2'][y_idx][0]

        x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

        p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

        p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

        T_uniq = np.unique(T_UNIFAC)
        for i in range(len(T_uniq)):
            # Plot median prediction
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT
            if j == 0:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b', label='Mean MC - No clusters')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r', label='Mean MC - With clusters')
            else:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], c='g', label='UNIFAC')

        # Scatter plot for experimental data
        ax.scatter(x_y, T_y, yy, c='k', marker='.', s=100, label='Experimental Data')

        # Custom legend
        custom_lines = [
            Line2D([0], [0], color='k', marker='.', linestyle='None', markersize=10),  # Experimental Data
            Line2D([0], [0], color='g', lw=4),  # UNIFAC
            Line2D([0], [0], color='b', lw=4), # Mean MC
            Line2D([0], [0], color='r', lw=4)
        ]

        ax.legend(custom_lines, ['Experimental', 'UNIFAC', 'MC - No cluster', 'MC - With clusters'], loc='upper left', bbox_to_anchor=(1.03, 1))

        ax.set_xlabel('Composition of component 1 [mol//mol]', fontsize=14)
        ax.set_ylabel('Temperature [K]', fontsize=14)
        ax.set_zlabel('Excess Enthalpy [J/mol]', fontsize=14, labelpad=10)
        ax.set_title(f'(1) {c1} + (2) {c2}', fontsize=20)
        plt.tight_layout()  # Adjust layout to make room for the legend

        plt.savefig(f'{path}/{j}.png', dpi=300)

        plt.close(fig)
        clear_output(wait=False)

In [5]:
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/2D_plots/Training'
excel_UNI_sheet = 'Training_Plots'
Idx_known = np.array(json.load(open(post_obj.data_file, 'r'))['Idx_known']) - 1
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')
    
all_mix = np.char.add(np.char.add(rec_dict_c_0['c1'].astype(str), ' + '), rec_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(rec_dict_c_0['c1'], ' + '), rec_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    y_idx = exp_mix == unique_mix[j]
    UNIFAC_idx = UNIFAC_mix == unique_mix[j]
    yy = rec_dict_c_0['y_exp'][y_idx]
    yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    x_y = rec_dict_c_0['x'][y_idx]
    T_y = rec_dict_c_0['T'][y_idx]
    c1 = rec_dict_c_0['c1'][y_idx][0]
    c2 = rec_dict_c_0['c2'][y_idx][0]

    x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

    p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

    p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

    T_uniq = np.unique(T_UNIFAC)
    for i in range(len(T_uniq)):
        if not os.path.exists(f'{path}/{j}_{i}.png'):
            print(f'{j+1} out of {len(unique_mix)} mixtures')
            print(f'{i+1} out of {len(T_uniq)} temperatures')
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT

            fig, ax = plt.subplots()
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], '-g', label='UNIFAC')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], '-b', label='Mean MC - No clusters')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], '-r', label='Mean MC - With clusters')
            ax.plot(x_y[T_y_idx], yy[T_y_idx], '.k', label='Experimental Data')
            ax.set_xlabel('Composition of Compound 1 [mol/mol]')
            ax.set_ylabel('Excess Enthalpy [J/mol]')
            ax.set_title(f'(1) {c1} + (2) {c2} at {T_uniq[i]:.2f} K')
            ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            fig.savefig(f'{path}/{j}_{i}.png', dpi=300)
            plt.close(fig)
            clear_output(wait=False)

In [6]:
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/3D_plots/Training'
excel_UNI_sheet = 'Training_Plots'
Idx_known = np.array(json.load(open(post_obj.data_file, 'r'))['Idx_known']) - 1
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(rec_dict_c_0['c1'].astype(str), ' + '), rec_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(rec_dict_c_0['c1'], ' + '), rec_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    if not os.path.exists(f'{path}/{j}.png'):
        print(f'{j+1} out of {len(unique_mix)} mixtures')
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        y_idx = exp_mix == unique_mix[j]
        UNIFAC_idx = UNIFAC_mix == unique_mix[j]
        yy = rec_dict_c_0['y_exp'][y_idx]
        yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        x_y = rec_dict_c_0['x'][y_idx]
        T_y = rec_dict_c_0['T'][y_idx]
        c1 = rec_dict_c_0['c1'][y_idx][0]
        c2 = rec_dict_c_0['c2'][y_idx][0]

        x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

        p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

        p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

        T_uniq = np.unique(T_UNIFAC)
        for i in range(len(T_uniq)):
            # Plot median prediction
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT
            if j == 0:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b', label='Mean MC - No clusters')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r', label='Mean MC - With clusters')
            else:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], c='g', label='UNIFAC')

        # Scatter plot for experimental data
        ax.scatter(x_y, T_y, yy, c='k', marker='.', s=100, label='Experimental Data')

        # Custom legend
        custom_lines = [
            Line2D([0], [0], color='k', marker='.', linestyle='None', markersize=10),  # Experimental Data
            Line2D([0], [0], color='g', lw=4),  # UNIFAC
            Line2D([0], [0], color='b', lw=4), # Mean MC
            Line2D([0], [0], color='r', lw=4)
        ]

        ax.legend(custom_lines, ['Experimental', 'UNIFAC', 'MC - No cluster', 'MC - With clusters'], loc='upper left', bbox_to_anchor=(1.03, 1))

        ax.set_xlabel('Composition of component 1 [mol//mol]', fontsize=14)
        ax.set_ylabel('Temperature [K]', fontsize=14)
        ax.set_zlabel('Excess Enthalpy [J/mol]', fontsize=14, labelpad=10)
        ax.set_title(f'(1) {c1} + (2) {c2}', fontsize=20)
        plt.tight_layout()  # Adjust layout to make room for the legend

        plt.savefig(f'{path}/{j}.png', dpi=300)

        plt.close(fig)
        clear_output(wait=False)

In [67]:
all_err_metrics = err_metrics_c_0.copy()
all_cols = all_err_metrics.columns.to_list()
apppend_cols = []
for i in range(len(all_cols)):
    if all_cols[i][0] == 'MC':
        t = all_cols[i][1]
        all_cols[i] = ('MC - No Cluster', t)
        apppend_cols += [('MC - With Cluster', t)]
all_err_metrics.columns = pd.MultiIndex.from_tuples(all_cols)
all_err_metrics = pd.concat([all_err_metrics,  pd.DataFrame({apppend_cols[i]: err_metrics_c_1.iloc[:,-3:].to_numpy()[:,i] for i in range(3)})], axis=1)

In [68]:
all_rec_err_metrics = rec_err_metrics_c_0.copy()
all_cols = all_rec_err_metrics.columns.to_list()
apppend_cols = []
for i in range(len(all_cols)):
    if all_cols[i][0] == 'MC':
        t = all_cols[i][1]
        all_cols[i] = ('MC - No Cluster', t)
        apppend_cols += [('MC - With Cluster', t)]
all_rec_err_metrics.columns = pd.MultiIndex.from_tuples(all_cols)
all_rec_err_metrics = pd.concat([all_rec_err_metrics,  pd.DataFrame({apppend_cols[i]: rec_err_metrics_c_1.iloc[:,-2:].to_numpy()[:,i] for i in range(2)})], axis=1)

In [69]:
all_err_metrics

IUPAC                 Number of known mixtures              \
        Component 1     Component 2              Component 1 Component 2   
0    Methanoic acid   Ethanoic acid                       10          26   
1    Methanoic acid  Propanoic acid                       10          24   
2    Methanoic acid   Butanoic acid                       10           3   
3    Methanoic acid         Benzene                       10          56   
4    Methanoic acid         Toluene                       10          40   
..              ...             ...                      ...         ...   
208      1-Propanol       1-Nonanol                       48          20   
209       1-Butanol      2-Propanol                       44          22   
210      1-Pentanol      2-Propanol                       32          22   
211       1-Hexanol      2-Propanol                       37          22   
212         Overall                                                        

    Number of common mixture Number of datapoints       UNIFAC               \
                                                           MAE         RMSE   
0                         10                  102   796.635367   840.928466   
1                         10                   54  1108.292486  1175.314991   
2                          0                   17  1064.724520  1165.069576   
3                          6                   53   147.074132   260.391836   
4                          5                   48   146.824259   253.702461   
..                       ...                  ...          ...          ...   
208                       19                    8    78.281591    82.436387   
209                       16                   38    18.134740    19.351540   
210                       11                   35    10.826704    11.451331   
211                       13                   11    41.774947    44.516866   
212                                          4890   161.614290   278.871619   

              MC - No Cluster                        MC - With Cluster  \
         MARE             MAE         RMSE      MARE               MAE   
0    8.125925      122.236453   128.108930  1.270185       2597.742218   
1    2.574847      326.861110   358.336269  0.758250        606.170337   
2    2.171473      237.790735   253.410872  0.523156        478.765865   
3    0.297583     2007.295839  2733.035298  6.971378       1759.386084   
4    0.409850     1138.277802  1640.645394  4.965447       1108.404122   
..        ...             ...          ...       ...               ...   
208  0.547346       46.760433    52.134085  0.328317         51.205870   
209  1.317811       15.175362    16.249295  1.102971         29.071353   
210  0.430612       41.833336    44.570988  1.630341         35.189962   
211  0.658538       40.703802    43.641633  0.641766         29.170356   
212  0.433465      662.360740  1620.031104  2.335915        866.308001   

                             
            RMSE       MARE  
0    2944.526092  30.299235  
1     923.335049   1.676497  
2     525.541215   0.960092  
3    2268.735187  13.009361  
4    1486.177756  10.298037  
..           ...        ...  
208    54.443612   0.358837  
209    30.999835   2.116728  
210    38.054189   1.372036  
211    31.329632   0.468153  
212  2941.560295   2.684733  

[213 rows x 15 columns]

In [70]:
all_rec_err_metrics

IUPAC               Number of known mixtures              \
        Component 1   Component 2              Component 1 Component 2   
0    Methanoic acid      Methanol                       10          41   
1    Methanoic acid       Ethanol                       10          41   
2    Methanoic acid    1-Propanol                       10          48   
3    Methanoic acid     1-Butanol                       10          44   
4    Methanoic acid    1-Pentanol                       10          32   
..              ...           ...                      ...         ...   
777       1-Octanol     1-Decanol                       35          36   
778       1-Decanol    2-Propanol                       36          22   
779       1-Decanol     2-Butanol                       36          13   
780       1-Decanol  tert-Butanol                       36          25   
781         Overall                                                      

    Number of common mixture Number of datapoints      UNIFAC              \
                                                          MAE        RMSE   
0                          7                   20  378.665000  556.145745   
1                          6                    8  130.201734  138.464460   
2                          8                    8  321.813760  345.460111   
3                          6                   10  373.480074  408.469181   
4                          6                    9  462.559492  499.424276   
..                       ...                  ...         ...         ...   
777                       30                   16   12.751365   14.005837   
778                        9                   20  150.334773  162.813298   
779                        5                   18  133.606846  147.655775   
780                       11                   20   12.543051   14.369671   
781                                         28197   84.036656  137.778923   

    MC - No Cluster             MC - With Cluster              
                MAE        RMSE               MAE        RMSE  
0        310.620564  453.487530        197.326074  318.819782  
1        211.642144  232.975956         32.270536   38.870173  
2         29.590808   44.690345         33.781065   43.477757  
3         31.491768   44.439938         18.049959   20.210290  
4         30.060552   36.316029         13.592397   16.198099  
..              ...         ...               ...         ...  
777        0.552341    0.640663          0.485705    0.614961  
778       49.667117   56.472583         14.953987   17.107702  
779       16.325413   18.348998          5.923562    6.984746  
780        0.181995    0.232998          2.373216    2.726481  
781       49.420201   82.869075         50.460890   79.700975  

[782 rows x 12 columns]

In [71]:
all_rec_err_metrics

IUPAC               Number of known mixtures              \
        Component 1   Component 2              Component 1 Component 2   
0    Methanoic acid      Methanol                       10          41   
1    Methanoic acid       Ethanol                       10          41   
2    Methanoic acid    1-Propanol                       10          48   
3    Methanoic acid     1-Butanol                       10          44   
4    Methanoic acid    1-Pentanol                       10          32   
..              ...           ...                      ...         ...   
777       1-Octanol     1-Decanol                       35          36   
778       1-Decanol    2-Propanol                       36          22   
779       1-Decanol     2-Butanol                       36          13   
780       1-Decanol  tert-Butanol                       36          25   
781         Overall                                                      

    Number of common mixture Number of datapoints      UNIFAC              \
                                                          MAE        RMSE   
0                          7                   20  378.665000  556.145745   
1                          6                    8  130.201734  138.464460   
2                          8                    8  321.813760  345.460111   
3                          6                   10  373.480074  408.469181   
4                          6                    9  462.559492  499.424276   
..                       ...                  ...         ...         ...   
777                       30                   16   12.751365   14.005837   
778                        9                   20  150.334773  162.813298   
779                        5                   18  133.606846  147.655775   
780                       11                   20   12.543051   14.369671   
781                                         28197   84.036656  137.778923   

    MC - No Cluster             MC - With Cluster              
                MAE        RMSE               MAE        RMSE  
0        310.620564  453.487530        197.326074  318.819782  
1        211.642144  232.975956         32.270536   38.870173  
2         29.590808   44.690345         33.781065   43.477757  
3         31.491768   44.439938         18.049959   20.210290  
4         30.060552   36.316029         13.592397   16.198099  
..              ...         ...               ...         ...  
777        0.552341    0.640663          0.485705    0.614961  
778       49.667117   56.472583         14.953987   17.107702  
779       16.325413   18.348998          5.923562    6.984746  
780        0.181995    0.232998          2.373216    2.726481  
781       49.420201   82.869075         50.460890   79.700975  

[782 rows x 12 columns]

In [72]:
with pd.ExcelWriter('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Errors_Metrics.xlsx') as writer:
    round_all_err_metrics = all_err_metrics.copy()
    round_all_err_metrics.iloc[:, -9:] = round_all_err_metrics.iloc[:, -9:].round(2)
    round_all_err_metrics.to_excel(writer, sheet_name='Testing')

    round_all_rec_err_metrics = all_rec_err_metrics.copy()
    round_all_rec_err_metrics.iloc[:, -6:] = round_all_rec_err_metrics.iloc[:, -6:].round(2)
    round_all_rec_err_metrics.to_excel(writer, sheet_name='Training')